<a href="https://colab.research.google.com/github/ArshHp/LLM/blob/main/Intent_Classifier_POC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install -q transformers nltk

In [27]:
import torch
import nltk
import re
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer

# Load a pre-trained model for text classification (DistilBERT fine-tuned on NLI)
MODEL_NAME = "facebook/bart-large-mnli"  # You can replace with a fine-tuned model
classifier = pipeline("zero-shot-classification", model=MODEL_NAME)

Device set to use cpu


In [32]:
# Ensure you have the necessary NLTK tokenizer
# Download punkt_tab
nltk.download('punkt_tab')
# Download the 'punkt' resource as well, if you haven't already.
nltk.download('punkt')

# Define a set of common greetings (expand as needed based on initial testing)
GREETING_PATTERNS = re.compile(r"^(hi|hello|hey|good morning|good evening|greetings|howdy|what's up|Good Eve)[,!\. ]*", re.IGNORECASE)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [30]:
def remove_greeting(user_input: str) -> str:
    """
    Removes greeting words if they appear at the beginning of the message.
    """
    # Remove greetings appearing at the start of the message
    cleaned_input = re.sub(GREETING_PATTERNS, "", user_input, count=1).strip()

    return cleaned_input if cleaned_input else None  # Return None if no meaningful content remains

def filter_greetings(user_input: str) -> str:
    """
    Processes user input: removes greetings and keeps only non-greeting content.
    """
    # Step 1: Remove greeting if it appears at the beginning
    cleaned_input = remove_greeting(user_input)

    if not cleaned_input:
        return None  # Entire message was a greeting

    # Step 2: Perform sentence-wise classification
    sentences = nltk.sent_tokenize(cleaned_input)
    candidate_labels = ["greeting", "non-greeting"]

    filtered_sentences = [
        sentence for sentence in sentences if classifier(sentence, candidate_labels)["labels"][0] == "non-greeting"
    ]

    return " ".join(filtered_sentences) if filtered_sentences else None


In [33]:
# Example Usage
user_messages = [
    "Hi, How are you doing? I need more details on OT products, please suggest.",
    "Good morning! Can you explain RAG pipelines?",
    "Hey there!",
    "Hello, what's the latest update on AI trends?",
    "Hello!",
    "How are you?",
    "Tell me about the latest trends in AI.",
    "Good morning!",
    "What is RAG in LLMs?",
    "Hello, tell me more about Service level agreement",
    "Hi,I need some more details on OpenText products, please suggest.",
    "Good Afternoon Sir, How are you?",
    "Good Eve,tell me what kind of inputs should I add here ?"
]

for msg in user_messages:
    processed_msg = filter_greetings(msg)
    if processed_msg:
        print(f"Sending to RAG: {processed_msg}")
    else:
        print(f"Ignored greeting: {msg}")

Sending to RAG: I need more details on OT products, please suggest.
Sending to RAG: Can you explain RAG pipelines?
Ignored greeting: Hey there!
Sending to RAG: what's the latest update on AI trends?
Ignored greeting: Hello!
Ignored greeting: How are you?
Sending to RAG: Tell me about the latest trends in AI.
Ignored greeting: Good morning!
Sending to RAG: What is RAG in LLMs?
Sending to RAG: tell me more about Service level agreement
Sending to RAG: I need some more details on OpenText products, please suggest.
Ignored greeting: Good Afternoon Sir, How are you?
Sending to RAG: tell me what kind of inputs should I add here ?
